# Projeto Aplicado IV — Dataset Overview
**Tema:** Previsão de Consumo de Energia Elétrica no Brasil  
**Data:** 28/08/2025  

Este notebook apresenta um **overview rápido** do dataset que será utilizado no projeto (séries temporais de carga de energia diária do ONS).


## Sobre o dataset
- **Fonte:** Operador Nacional do Sistema Elétrico (ONS) — Dados Abertos  
- **Conjunto:** *Carga de Energia Diária* (CSV)  
- **Periodicidade:** diária  
- **Principais variáveis esperadas:** `data`, `subsistema`, `carga_mwmed` (nomes podem variar conforme versão)  
- **Repositório do projeto:** https://github.com/contateobrito/Projeto_Aplicado_IV

> Observação: Baixe o CSV correspondente (ex.: `Carga_Energia_2025.csv`) do portal do ONS e coloque-o na pasta `data/` do repositório local.


In [1]:
## Setup
# Bibliotecas principais
import os
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

print("Pandas:", pd.__version__)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
## Caminhos e arquivo de dados
# Ajuste o caminho conforme sua estrutura local do repositório
DATA_DIR = os.path.join(os.getcwd(), "data")
CSV_FILE = os.path.join(DATA_DIR, "Carga_Energia_2025.csv")  # exemplo; renomeie se necessário

print("Pasta de dados:", DATA_DIR)
print("Arquivo CSV esperado:", CSV_FILE)
print("Existe?", os.path.exists(CSV_FILE))


In [ ]:
## Leitura dos dados
df = None
if os.path.exists(CSV_FILE):
    try:
        df = pd.read_csv(CSV_FILE, encoding="utf-8")
    except Exception as e:
        print("Falha ao ler CSV com UTF-8:", e)
        print("Tentando 'latin-1'...")
        df = pd.read_csv(CSV_FILE, encoding="latin-1")
else:
    print("⚠️ CSV não encontrado. Coloque o arquivo na pasta 'data/' para continuar.")

df.head(10) if df is not None else None


In [ ]:
## Informações básicas
if df is not None:
    display(df.info())
    display(df.describe(include='all').T.head(20))
else:
    print("Sem dados carregados.")


## Padronização mínima de colunas (opcional)
if df is not None:
    cand_data = [c for c in df.columns if c.lower() in ("data", "dt", "date")]
    cand_subs = [c for c in df.columns if c.lower() in ("subsistema","subsis","area","regiao")]
    cand_carga = [c for c in df.columns if c.lower() in ("carga_mwmed","carga","mwmed","valor")]

    col_data = cand_data[0] if cand_data else None
    col_subs = cand_subs[0] if cand_subs else None
    col_carga = cand_carga[0] if cand_carga else None

    print("Mapeamento detectado:", col_data, col_subs, col_carga)

    if col_data is not None:
        df["data_padrao"] = pd.to_datetime(df[col_data], errors="coerce", dayfirst=True, infer_datetime_format=True)
    if col_subs is not None:
        df["subsistema_padrao"] = df[col_subs].astype(str)
    if col_carga is not None:
        df["carga_mwmed_padrao"] = pd.to_numeric(df[col_carga], errors="coerce")

    display(df.head(10))
else:
    print("Sem dados carregados.")


In [ ]:
## Resumo rápido
if df is not None:
    if "data_padrao" in df.columns:
        print("Período:", df["data_padrao"].min(), "→", df["data_padrao"].max())
    if "subsistema_padrao" in df.columns:
        print("Subsistemas:", sorted(df["subsistema_padrao"].dropna().unique().tolist())[:20])
    na_counts = df.isna().sum().sort_values(ascending=False).head(10)
    print("\nTop 10 colunas com ausências:\n", na_counts)
else:
    print("Sem dados carregados.")


In [ ]:
## Visualização simples (agregado diário)
if df is not None and all(col in df.columns for col in ["data_padrao","carga_mwmed_padrao"]):
    serie = df.dropna(subset=["data_padrao","carga_mwmed_padrao"]).groupby("data_padrao")["carga_mwmed_padrao"].sum()
    plt.figure(figsize=(10,4))
    serie.plot()
    plt.title("Carga diária agregada (MW médios)")
    plt.xlabel("Data")
    plt.ylabel("Carga (MW médios)")
    plt.tight_layout()
    plt.show()
else:
    print("Sem dados padronizados suficientes para o gráfico (data/carga).")


## Próximos passos
- Verificar dicionário de dados do ONS e confirmar nomes exatos das colunas.  
- Tratar valores ausentes e padronizar os tipos.  
- Definir escopo: nacional agregado ou por subsistema.  
- Preparar *notebooks* para EDA (Etapa 2) e para modelagem (Etapa 3).
